https://www.kaggle.com/tuckerarrants/text-generation-with-huggingface-gpt2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 7.1 MB/s 
     |████████████████████████████████| 59 kB 7.7 MB/s 
     |████████████████████████████████| 3.3 MB 94.2 MB/s 
     |████████████████████████████████| 895 kB 73.2 MB/s 
     |████████████████████████████████| 596 kB 81.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [ ]:
import re
with open('/content/drive/MyDrive/Datasets/all_rules.txt', encoding='utf8') as f:
    text = f.read()

text = re.sub('\n{2,}', '\n', text)
print(text[:100])

Количество игроков: от 2 до 10 Время игры: от 20 минут Задача: первым скинуть свои карты. На этом да


In [ ]:
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

13229


array([  684,   823,  2030, 12525,    30,   360,   491,   519,  1533,
        9661])

In [ ]:
l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

13229 10572 2643


In [ ]:
from transformers import GPT2LMHeadModel, AdamW

model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)


model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
batch_size = 8
max_len = 64
epochs = 50

n_train = len(train) // (batch_size * max_len)
n_test = len(test) // (batch_size * max_len)
print(n_train, n_test)

optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

20 5


In [ ]:
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                             token_type_ids=None, 
                            #  attention_mask=batch_mask,
                             labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print('epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/50 : training


  0%|          | 0/20 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/5 [00:00<?, ?it/s]

epoch 2/50 : training


...

  0%|          | 0/5 [00:00<?, ?it/s]

epoch 50/50 : training


  0%|          | 0/20 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
torch.save(model, '/content/drive/MyDrive/model.pt')

In [ ]:
  # https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate
# Model without training
import textwrap

# prompt = "print('Hello world!') "
prompt = 'Участники этой напряженной игры'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=5.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=3,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Участники этой напряженной игры уже несколько месяцев находились под домашним арестом и даже не представляли, какой
приговор может им вынести суд и куда отправят. В этот день их привезли из колонии № 6 на улице Советской, и уже утром 25
апреля их отправили в психушку № 2 для постоянного пребывания там в одиночке. В этом заведении они были почти ежедневно:
днем, когда работали, когда отдыхали (таких заключенных обычно отправляют по три, но обычно в течение полугода), когда
приходили домой и сидели, но не получали ничего; затем уезжали на месяц в лагерь; потом на три месяца опять на свободу и
на этот раз там их снова оставляли без содержания на два года или дольше. И, наконец, после окончания срока, они
возвращались сюда опять.
------------------
Участники этой напряженной игры, конечно, могли бы с ним согласиться. Если же и соглашаются, то все как один будут не
правы, если же они и правы и правы — им надо дать бой, ведь они — «свои»… Если все будет хорошо, пусть будут другие
проблемы. Н

In [ ]:
import textwrap

prompt = 'Участники этой напряженной игры'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=10,
    do_sample=True,
    temperature=5.,
    top_k=10,
    top_p=0.75,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Участники этой напряженной игры выбирают среди выложенных карточек лидеров одного из городов. Игрок должен выбрать самый
ужасный из предложенных маршрутов, затем перетасовать карты и перетащить к себе в машину. Если игрок не тянет карту, у
него отнимается возможность проголосовать. У игрока также есть возможность отказаться выполнять все команды, которые он
выбрал до этого (при необходимости можно заменить слова). Игра продолжается в обратном порядке: игроки передвигают свои
карточки по полю с помощью специальных рычажных устройств (ручек или тягл). В игре используются специальные карты
вагонов (самых обычных) и карты поездов дальнего следования. Каждый игрок получает карточку из руки с номером,
начинающимся на "ноль". После этого игрок может выкладывать свои карты на игровом поле
------------------
Участники этой напряженной игры попадают в «Хромого кота» — психологическое и игровое консультирование по вопросам любви
и ненависти. Каждый ход вы получаете по 10 писем от разных игроков, 

In [ ]:
def generate(prompt, len_gen=20, temperature=1):
    generated = tokenizer.encode(prompt)
    context = torch.tensor([generated]).to(device)
    past = None

    for i in tqdm(range(len_gen)):
        output, past = model(context, past_key_values=past).values()
        # token = torch.argmax(output[..., -1, :], dim=-1)
        output = output / temperature
        token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
        generated += token.tolist()
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    return sequence

In [ ]:
import textwrap


# prompt = """<!DOCTYPE html>
# <html>
# <body>

# <h1>My First Heading</h1>
# <p>My first paragraph.</p>"""
prompt = 'Лучшая игра на свете'
print(textwrap.fill(generate(prompt, 200, temperature=1), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

Лучшая игра на свете!» Вас ждут веселые игровые автоматы, о которых мы расскажем чуть ниже. Раздача роли происходит в
начале игры, остальные роли распределены следующим образом. Игроки берут кубики и выкладывают фишки на соответствующие
клетки. Затем все игроки начинают расселяться, размещая на одной клетке по пять карточек. Не забывайте оставлять в игре
свои игрушки, иначе вы можете нарваться на мошенников. Через некоторое время вы узнаете из этой новости, кому
принадлежат все эти дома и поселки. Теперь возьмите себе на заметку! Правила игры Подготовка к игре Собирайте бюллетени
свид-рианта и проверяйте население. Собирайте свежесобранные бюллетени и сдавайте их в Комитет Государственной Думы
Федерального Собрания Российской Федерации. Расставляйте «руки» — голосования, которые будут проверять население на
склонность к сифилису или к туберкулезу. Одновременно ведите телефонные провода вокруг каждого участка. Собравшиеся
бюллетени не должны попадать на руки людям с незарегистрированным

In [ ]:
prompt = 'Участники этой напряженной игры'
print(textwrap.fill(generate(prompt, 200, temperature=1), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

Участники этой напряженной игры постоянно обновляют свои поля или убирают их в особые тайники своего поля, чтобы однажды
утром по каким-то причинам им больше не удалось найти и перетащить на свой участок столько гадостей и несчастий,
обрушившихся на их благодетелей. Каждый ход один из участников становится победителем. Если команда Мэтта приносит на
свой участок больше всего гадостей и несчастий (3:1 или 4:2), она получает «Полный взрыв»! Ура, товарищи! Скорее вперед,
к победе во всех стратегических и коммерческих проектах! Основная задача любого игрока в этой игре — найти и перетащить
на свой участок столько интересных и необычных свойств и характеристик, что скулы сводит от зависти! Правила игры
«МАФИЯ» — самая популярная настольная игра в Америке. Правила игры «МАФИЯ» — это первый в мире шутер с уникальной
механикой настольного проектирования. Цель игры — исследовать население городов и стран с удивительной быстротуностью.
Цель игры — угадать, какие


In [ ]:
prompt = 'Игра на двух человек'
print(textwrap.fill(generate(prompt, 200, temperature=1), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

Игра на двух человек 4. Если у вас уже был «Мастер Иствик», откройте первую партию и совершите ход 3. Передвиньте фишку
персонажа на одну клетку вперед, чтобы проверить правильность выбора карточки. 4. Вы можете сыграть по очереди все
четыре вида карт. 5. Вы можете воспользоваться любой другой подходящей для хода картами, например, попросить игрока
перетащить ваш любимый фужер. 6. Игрок переходит на ваш участок и выполняет семявое количество действий, путем
перетаскивания своего же семени на клетку вперед. 7. Перед тем, как отнестись к игре со всей серьезностью, она может
показаться вам просто комедией. 16. Национальное управление расследует убийство, которого еще не было на карте казни.
Вот чем должен заниматься каждый игрок, чтобы унять кровь и обезопасить жизнь других игроков. На этот раз национальным
властям придется разбираться во всех убийствах на Земле, начиная с массовых убийств в начале 60-х годов и кончая
кровавыми столкновениями на пустынных


In [ ]:
prompt = 'Вы готовы?'
print(textwrap.fill(generate(prompt, 200, temperature=1), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

Вы готовы?&raquo; Кто обращается за содействием к Вам, должны прийти Ваши друзья или знакомые, чтобы проголосовать за
Вас. Если все соберутся достаточно большая куча, чтобы разойтись, то все игроки по очереди поднимаются по клеткам на
одну клетку вперед. Когда все расставлены, игра завершается. Порядок хода Каждый ход игрок перемещает фишку персонажа с
клетки на клетку, начиная с самого крайнего игрока. Затем он останавливается и выставляет свою фишку на игровое поле.
Устанавливаются игровые автоматы &laquo;Активити&raquo; и &laquo;Полисоактивный&raquo;. Автомат показывает определенное
число карточек с символом плодородия, и по мере своего хода выбирает из них одну особенную для голосования. В комплект
входит 10 карточек с кодовым именем и соответствующий символ. Каждая карточка содержит одну особую карточку, которая
указывает на то животное, на которое она собственно поставлена. Если какое-либо животное отсутствует в игре


In [ ]:
prompt = 'В случае проигрыша'
print(textwrap.fill(generate(prompt, 200, temperature=1), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

В случае проигрыша Постройте Волшебную избушку. Поставьте на клетку Игрок должен построить Избушку Волшебной страны,
чтобы ему взяли Рис Волшебной страны (Рисуют на нем маркером с игрового поля). Постройте Волшебную избушку из картона и
обжмите розгой. Когда Вы обуйте Розга, она может потерять ориентацию. Когда Её обмотаете верните на клетку назад.
Создание Волшебной избушки Избушка Волшебной страны — популярная карточка в англоязычных странах. Игроки могут купить
Участок Волшебной страны в двух поселениях. Досталась каждому поселению по одной игровой клетке. 16 участников делятся
на три команды (две команды по три игрока) и из колоды карт выкладывается карта, на которой отмечаются символы страны.
Каждая команда должна дать на карте такой символ, который бы соответствовал названию страны (Рисуют на картоне с
игрового поля). Это может быть слово, пословица, картинка. Если карта выпадет из колоды


In [ ]:
prompt = 'Это правила очень активной игры для дружной компании'
print(textwrap.fill(generate(prompt, 200, temperature=1), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

Это правила очень активной игры для дружной компании или для соревнований с друзьями. Правила игры для трех игроков
Каждый ход вы бросаете по одной карте. Досталась же карта случая Игроки бросают по одной карте случайного свойства,
чтобы было легче найти общий язык. Угадывание карты Следующий ход вы играете по часовой стрелке. Вам подсказывают
случайные карты двух стран. То есть если какая-либо страна не угадана раньше, чем вы посмотрите на карту, то она уже
угадана второй стороной. Если где-то она «добыта» второй стороной, то вы впервые видите страну. Как вас звать Игрок,
изображенный на карте, показывает, какое животное он загадал. Если животное нет в поле, он называет его своим именем и
показывает, на кого он указывает. Если он угадал, то показывает эту карту своему соседу, и так повторяется 5 раз. Если
он угадал сразу несколько карт, то берет из этой карты одну, а остальные бросает в сторону. Как играть С каждым днем все
больше любителей играть в


In [ ]:
prompt = 'В начале'
print(textwrap.fill(generate(prompt, 200, temperature=.8), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

В начале игры вы получаете карточку с номером нужного цвета. Ход игры Игра продолжается, пока не останется только один
участник. Если все игроки спасовали, то начинается следующий ход. В начале игры каждый игрок получает по четыре карточки
цвета «семерка» с номером, соответствующим цвету его карты «третий ход». В начале игры все игроки достают из колоды 4
карты «третий ход». Когда все 4 карты «третий ход» подойдут к концу, то все четыре карты «третий ход» убираются в
коробку с картами «семерки» и размещаются в центре стола лицом вверх. Завершается игра посещением Храма. После окончания
игры все игроки расходятся по своим комнатам. Каждый из них выбирает себе самого красивого. После чего все игроки
выбирают себе самого опасного. Подробнее об игре «Шакал» в картинках Вы найдете подробную информацию о том, как играть в
«Шакала» прямо сейчас. Вам не нужно покидать дом или даже специально покидать его. Вам достаточно зайти в свой аккаунт


In [ ]:
prompt = 'Каждый игрок приходит со своим питомцем'
print(textwrap.fill(generate(prompt, 200, temperature=.8), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

Каждый игрок приходит со своим питомцем и кладет его на клетку с игрой. Игра продолжается до тех пор, пока один из
игроков не наберет достаточное количество очков, чтобы пройти в следующий раунд. Партию начинают все игроки, идущие
первым. Каждый игрок выбирает себе по пять животных. Как только игрок соберет достаточное количество очков, чтобы пройти
в следующий раунд, он снова участвует в игре. «Перехват» — игра, где нужно перехватить два кубика. Правила игры для двух
игроков просты: игрок берет кубик и делает одно движение по часовой стрелке, после чего бросает кубик на клетку с игрой.
Клетка с игрой остается стоять на месте до тех пор, пока один из игроков не наберет нужное количество очков, чтобы
пройти в следующий раунд. «Перехват» — игра для двух игроков, в которой нужно перехватить два кубика. Цель игры —
перехватить у одного игрока два кубика и заставить их переворачиваться. Цель игры — определить, кто из игроков может
перехватить один кубик. Для этого игрок переворачивает верхн

In [ ]:
prompt = 'Килька'
print(textwrap.fill(generate(prompt, 200, temperature=.8), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

Килька в банке» — эта увлекательная игра позволит узнать, какая из рас загадана, и с какой можно сбежать, чтобы избежать
неприятных сюрпризов. Каждый ход вы будете проигрывать определенную карту, и если она будет выбрана, то команда, которой
она воспользовалась, сможет спастись. Правила игры Быстро в картинках 3. Выложите карточки для голосования. Каждый ход
вы бросаете по одной карте. Это нужно, чтобы понять, какая из рас угадана верно, и как поступить в случае, если игрок
ошибся. 4. Игрок раскрывает карты и показывает, какая из них выбрана верно. Когда игрок произнесет слово, он показывает
карточку, на которой указано число. 5. Когда игрок произнесет слово, он показывает карточку, на которой указан символ
«Возьми иглу». 6. Когда игрок произнесет слово, он показывает карточку, на которой указан символ «Вперед». 7. Когда
игрок произнесет слово, он показывает карточку, на которой указан символ «Отойди в сторону». 8


In [ ]:
prompt = 'Наруто'
print(textwrap.fill(generate(prompt, 200, temperature=.8), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

Наруто" — это уже по новеллам. Здесь вам предлагается исследовать различные локации, сражаться и даже обмениваться
телефонами. В то же время в этой игре действуют специальные правила: «Подложи товарищу» — игроки поочередно рассказывают
друг другу, что происходит в их новой локации. Игроки же получают информацию из первых чисел карт метро, на которые они
попали. «Используй свой телефон» — при игре «Используй свой телефон» игроки должны в течение одной минуты придумать, как
сделать так, чтобы у всех игроков на руках было хотя бы по одному телефону. Подготовка к игре В первую игруратсу вас
будет вести специальный игрок. Он будет вести игру по часовой стрелке, а затем по часовой. Чтобы сделать свой ход, игрок
должен в течение одной минуты придумать, как лучше использовать телефон. Если у вас еще нет ни одной свободной минуты,
то вы всегда можете поменять часовой стрелкой направление своего хода. Например, если у вас есть только пара минут,
чтобы определиться с тем, какой из


In [ ]:
prompt = 'Оксимирон'
print(textwrap.fill(generate(prompt, 200, temperature=.8), 120))

  0%|          | 0/200 [00:00<?, ?it/s]

Оксимирон» — стратегическая компьютерная игра, разработанная и выпущенная компанией «АйТи» в 2001 году. Проходил конкурс
«Европейский Экспресс», в котором принимал участие 23 команды из 13 стран. В игре действуют специальные правила: игрок
должен выбрать интересующую его карту, а затем, дождавшись своей очереди, загадать соответствующее слово. После чего
команда должна найти и выложить на стол выбранную карту. Правила игры В игре есть специальный символ «Рисунки». Рисунки,
размещенные на обратной стороне карты, показывают уровень развития игроков. На первой карте показывается уровень
развития игрока. На второй карте — уровень развития команды. На третьей карте — уровень развития команды. Игрок должен
представить несколько карт, находящихся между собой. Например, если на первой карте нарисована «Хищная обезьяна», то на
ней нарисована «Самая опасная обезьяна на свете». Другая карта, на которой изображена «Самая ядовитая змея», показывает
уровень развития команды. Третья карта, на которой